In [ ]:
import functools

import optax
import jax.numpy as jnp
import jax
import e3nn_jax as e3nn
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors
import plotly.graph_objects as go

import sys
sys.path.append('../')

In [ ]:
from symphony import loss
from symphony import models

In [ ]:
target_positions = jnp.asarray([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
# target_positions = jnp.asarray([[1., 0., 0.]])
target_position_inverse_temperature = 10000.
lmax = 3
res_beta = 40
res_alpha = 39

In [ ]:
log_true_angular_coeffs = jax.vmap(
    lambda pos: loss.target_position_to_log_angular_coeffs(
        pos, target_position_inverse_temperature=target_position_inverse_temperature, lmax=lmax,
    )
)(target_positions)

compute_grid_of_joint_distribution_fn = functools.partial(
        models.compute_grid_of_joint_distribution,
        res_beta=res_beta,
        res_alpha=res_alpha,
        quadrature="soft",
    )
true_angular_dist = jax.vmap(
    compute_grid_of_joint_distribution_fn,
)(jnp.ones((target_positions.shape[0], 1)), log_true_angular_coeffs)
true_angular_dist.grid_values = true_angular_dist.grid_values[:, 0, :, :]
mean_true_angular_dist = e3nn.SphericalSignal(
    grid_values=true_angular_dist.grid_values.mean(axis=0),
    quadrature=true_angular_dist.quadrature
)
mean_true_angular_dist


In [ ]:
def coeffs_to_dist(log_predicted_angular_coeffs):
    predicted_angular_dist = compute_grid_of_joint_distribution_fn(jnp.ones((1,)), log_predicted_angular_coeffs)
    predicted_angular_dist = predicted_angular_dist[0, :, :]
    return predicted_angular_dist


def coeffs_to_logits(log_predicted_angular_coeffs):
    predicted_angular_dist = coeffs_to_dist(log_predicted_angular_coeffs)
    predicted_angular_logits = predicted_angular_dist.apply(models.safe_log)
    return predicted_angular_logits

In [ ]:
def factorized_position_loss(coeffs, dist) -> jnp.ndarray:
    """Computes the loss over position probabilities using separate losses for the radial and the angular components."""
    # Radial loss is simply the negative log-likelihood loss.
    # loss_radial = -preds.globals.radial_logits.sum(axis=-1)
    loss_radial = 0.

    predicted_angular_logits = coeffs_to_logits(coeffs)
    # The angular loss is the KL divergence between the predicted and the true angular distributions.
    res_beta, res_alpha = (
        predicted_angular_logits.res_beta,
        predicted_angular_logits.res_alpha,
    )

    # jax.debug.print("max={x}", x=true_angular_dist.grid_values.max())
    # jax.debug.print("min={x}", x=true_angular_dist.grid_values.min())
    assert predicted_angular_logits.shape == (
        res_beta,
        res_alpha,
    ), (predicted_angular_logits.shape, dist.shape)

    loss_angular = loss.kl_divergence_on_spheres(
        dist, predicted_angular_logits
    )

    loss_position = loss_radial + loss_angular
    return loss_position


In [ ]:
@jax.jit
def loss_fn(coeffs, target_dist):
    return factorized_position_loss(coeffs, target_dist).mean()

@functools.partial(jax.jit, static_argnames=("tx", "use_mean_dist",))
def step_fn(rng, coeffs, opt_state, tx, use_mean_dist):
    if use_mean_dist:
        target_dist = mean_true_angular_dist
    else:
        step_rng, rng = jax.random.split(rng)
        target_index = jax.random.choice(step_rng, a=target_positions.shape[0])
        target_dist = true_angular_dist[target_index]

    grads = jax.grad(loss_fn)(coeffs, target_dist)
    loss_val = loss_fn(coeffs, mean_true_angular_dist)
    updates, opt_state = tx.update(grads, opt_state)
    coeffs = optax.apply_updates(coeffs, updates)
    return rng, coeffs, opt_state, loss_val

In [ ]:
all_losses_by_hparams = {}
all_steps_by_hparams = {}
all_coeffs_by_hparams = {}

for use_mean_dist in [True, False]:
    for learning_rate in [1e1, 1e0, 1e-1, 1e-2, 1e-3, 1e-4]:
        
        rng = jax.random.PRNGKey(0)
        init_coeffs = e3nn.normal(e3nn.s2_irreps(lmax=lmax), rng)
        tx = optax.adam(learning_rate)
        opt_state = tx.init(init_coeffs)

        coeffs = init_coeffs
        loss_val = float(loss_fn(coeffs, mean_true_angular_dist))

        all_coeffs = []
        all_steps = []
        all_losses = []
        for step in range(10000):
            if step % 1000 == 0:
                all_coeffs.append(coeffs)
                all_steps.append(step)
                all_losses.append(float(loss_val))

            rng, coeffs, opt_state, loss_val = step_fn(rng, coeffs, opt_state, tx, use_mean_dist)
            if step == 10000 - 1:
                print(f"step={step}: loss={loss_val}")

        all_losses_by_hparams[(use_mean_dist, learning_rate)] = all_losses
        all_steps_by_hparams[(use_mean_dist, learning_rate)] = all_steps
        all_coeffs_by_hparams[(use_mean_dist, learning_rate)] = all_coeffs

In [ ]:
sns.set_style("darkgrid")
fig, ax = plt.subplots(ncols=2, figsize=(10, 6), sharey=True)
for learning_rate in [1e0, 1e-1, 1e-2, 1e-3, 1e-4]:
    sns.lineplot(x=all_steps_by_hparams[(False, learning_rate)], y=all_losses_by_hparams[(False, learning_rate)],
                 ax=ax[0], hue=learning_rate, hue_norm=matplotlib.colors.LogNorm(vmin=1e-5, vmax=1e1), palette="viridis")
    sns.lineplot(x=all_steps_by_hparams[(True, learning_rate)], y=all_losses_by_hparams[(True, learning_rate)],
                 ax=ax[1], hue=learning_rate, hue_norm=matplotlib.colors.LogNorm(vmin=1e-5, vmax=1e1), palette="viridis")

fig.suptitle(f"Training Curves for Different Learning Rates")
ax[0].set_title("Training on Randomly Sampled Angular Distributions")
ax[1].set_title("Training on Mean Angular Distribution")
ax[0].set_xlabel("Steps")
ax[0].set_ylabel("Loss")
ax[1].set_xlabel("Steps")
ax[1].set_ylabel("Loss")
ax[0].set_yscale("log")
plt.legend(title="Learning Rate")
plt.show()
    

In [ ]:
use_mean_dist = True
learning_rate = 1e-2

all_steps = all_steps_by_hparams[(use_mean_dist, learning_rate)]
all_coeffs = all_coeffs_by_hparams[(use_mean_dist, learning_rate)]
# Make a slider plot of the angular distributions.
fig = go.Figure()
for step, coeffs in zip(all_steps, all_coeffs):
    step_dist = coeffs_to_dist(coeffs)
    step_trace = go.Surface(
        step_dist.plotly_surface(),
        colorscale=[[0, "rgba(4, 59, 192, 0.)"], [1, "rgba(4, 59, 192, 1.)"]],
        showscale=True,
        cmin=step_dist.grid_values.min().item(),
        cmax=step_dist.grid_values.max().item(),
        visible=False,
    )
    fig.add_trace(step_trace)

# Make 0th trace visible
fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": f"Step: {all_steps[i]}"}],  # layout attribute
        label=f"{all_steps[i]}"
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Step: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()
